In [2]:
import pandas as pd

In [119]:
xcel = "Experiment/SMOTERegressionCases.xlsx"

# Load each sheet separately
df1 = pd.read_excel(xcel, sheet_name='Seed410_thr0.77_perc0.2')
df2 = pd.read_excel(xcel, sheet_name='Seed527_thr0.77_perc0.2')
df3 = pd.read_excel(xcel, sheet_name='Seed222_thr0.77_perc0.2')
df4 = pd.read_excel(xcel, sheet_name='Seed326_thr0.77_perc0.2')
df5 = pd.read_excel(xcel, sheet_name='Seed832_thr0.77_perc0.2')

# ## dropping column X, as it is unwanted
# df1 = df1.drop(['X'],axis=1)
# df2 = df2.drop(['X'],axis=1)
# df3 = df3.drop(['X'],axis=1)
# df4 = df4.drop(['X'],axis=1)
# df5 = df5.drop(['X'],axis=1)

In [120]:
df = pd.read_csv("Datasets/new.csv")
df = df.drop("Unnamed: 0",axis=1)
df

,id,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,1,21,74,1,166.911029,80.265922,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,5,61,0,154.900000,73.600000,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,7,58,0,166.911029,80.265922,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,9,94,1,182.800000,66.200000,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,9,91,1,166.911029,80.265922,1.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,515,2,56,0,165.000000,72.600000,0.0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
502,516,13,74,0,166.911029,55.000000,0.0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
503,517,19,81,1,158.000000,65.000000,0.0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,519,9,74,1,166.911029,81.700000,0.0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Function to plot histogram of original dataset and the synthetic dataset

In [121]:
import plotly.express as px

def plot_distribution(dataframe, column_name, description=None):
    try:
        # Create a Plotly distribution plot with black borders
        fig = px.histogram(dataframe, x=column_name, nbins=10, marginal="box", 
                           title=f"Distribution of {column_name}" if description is None else description, 
                           histnorm='density')

        # Customize the histogram bars to add black borders
        fig.update_traces(marker=dict(line=dict(color='black', width=1)))

        # Show the plot
        fig.show()
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to plot the distribution with a custom description
plot_distribution(df, 'hospital_length_of_stay', "Hospital Length of Stay Distribution (Original Dataset)")
plot_distribution(df1, 'hospital_length_of_stay', "Hospital Length of Stay Distribution (Seed410_thr0.77_perc0.2)")
plot_distribution(df2, 'hospital_length_of_stay', "Hospital Length of Stay Distribution (Seed527_thr0.77_perc0.2)")
plot_distribution(df3, 'hospital_length_of_stay', "Hospital Length of Stay Distribution (Seed222_thr0.77_perc0.2)")
plot_distribution(df4, 'hospital_length_of_stay', "Hospital Length of Stay Distribution (Seed326_thr0.77_perc0.2)")
plot_distribution(df5, 'hospital_length_of_stay', "Hospital Length of Stay Distribution (Seed832_thr0.77_perc0.2)")

### Function to bin the dataset and find the columns that do not have 0 as values in all the rows

In [122]:
def bin_hlos_and_create_summary_df(df):
    """
    Creates separate DataFrames based on specific bins for the 'hospital_length_of_stay' column,
    removes columns with all zeros, and returns a DataFrame summarizing the column names for each bin.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing 'hospital_length_of_stay' column.

    Returns:
    pd.DataFrame: A DataFrame summarizing the column names for each bin.
    """
    # Creating DataFrames for each bin
    df_10_20 = df[(df['hospital_length_of_stay'] >= 10) & (df['hospital_length_of_stay'] < 20)]
    df_20_30 = df[(df['hospital_length_of_stay'] >= 20) & (df['hospital_length_of_stay'] < 30)]
    df_30_40 = df[(df['hospital_length_of_stay'] >= 30) & (df['hospital_length_of_stay'] < 40)]
    df_40_50 = df[(df['hospital_length_of_stay'] >= 40) & (df['hospital_length_of_stay'] < 50)]
    df_50_60 = df[(df['hospital_length_of_stay'] >= 50) & (df['hospital_length_of_stay'] < 60)]
    df_60_70 = df[(df['hospital_length_of_stay'] >= 60) & (df['hospital_length_of_stay'] < 70)]
    df_70_80 = df[(df['hospital_length_of_stay'] >= 70) & (df['hospital_length_of_stay'] < 80)]

    # Removing columns with all zeros for each DataFrame
    df_10_20 = df_10_20.loc[:, (df_10_20 != 0).any()]
    df_20_30 = df_20_30.loc[:, (df_20_30 != 0).any()]
    df_30_40 = df_30_40.loc[:, (df_30_40 != 0).any()]
    df_40_50 = df_40_50.loc[:, (df_40_50 != 0).any()]
    df_50_60 = df_50_60.loc[:, (df_50_60 != 0).any()]
    df_60_70 = df_60_70.loc[:, (df_60_70 != 0).any()]
    df_70_80 = df_70_80.loc[:, (df_70_80 != 0).any()]

    # Creating a summary DataFrame for column names
    summary_data = {
        'Bin': ['10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80'],
        'Names': [
            df_10_20.columns.tolist(),
            df_20_30.columns.tolist(),
            df_30_40.columns.tolist(),
            df_40_50.columns.tolist(),
            df_50_60.columns.tolist(),
            df_60_70.columns.tolist(),
            df_70_80.columns.tolist()
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)

    return summary_df

In [123]:
# Create the summary DataFrame
summary_df = bin_hlos_and_create_summary_df(df)
summary_df1 = bin_hlos_and_create_summary_df(df1)
summary_df2 = bin_hlos_and_create_summary_df(df2)
summary_df3 = bin_hlos_and_create_summary_df(df3)
summary_df4 = bin_hlos_and_create_summary_df(df4)
summary_df5 = bin_hlos_and_create_summary_df(df5)

In [124]:
summary_df

,Bin,Names
0,10-20,"[id, hospital_length_of_stay, age, sex, height..."
1,20-30,"[id, hospital_length_of_stay, age, sex, height..."
2,30-40,"[id, hospital_length_of_stay, age, sex, height..."
3,40-50,"[id, hospital_length_of_stay, age, sex, height..."
4,50-60,"[id, hospital_length_of_stay, age, sex, height..."
5,60-70,"[id, hospital_length_of_stay, age, sex, height..."
6,70-80,"[id, hospital_length_of_stay, age, sex, height..."


### Figuring out the different column names

In [125]:
# Define the personal values based on the first 9 columns
personal = df_count_values[:9]

# Initialize lists for the categories
comorbidities = []
reason_for_admit = []
medications = []

# Classify comorbidities
for value in df_count_values[7:]:
    if value.startswith("reason_"):
        break  # Stop if we reach the first 'reason_' column
    else:
        comorbidities.append(value)  # Append to comorbidities

# Filter columns to classify reason for admit and medications
filtered_df_count_values = [
    value for value in df_count_values if value not in comorbidities and value not in personal
]

# Populate reason_for_admit and medications lists
for value in filtered_df_count_values:
    if value.startswith("reason_"):
        reason_for_admit.append(value)
    else:
        medications.append(value)

### Function to find the number of feature types for each bins, personal, medications, reason for admit or comorbidities

In [126]:
import pandas as pd

def check_value(df):
    """
    This function checks which values from various lists are present
    in the 'Names' column of the DataFrame and counts their occurrences.
    It adds new columns to the DataFrame for each category.

    Parameters:
    df (pd.DataFrame): Input DataFrame containing a 'Names' column.

    Returns:
    pd.DataFrame: Updated DataFrame with new count columns.
    """
    # Initialize lists for counts
    counts_personal = []
    counts_comorbidities = []
    counts_reason_admit = []
    counts_medications = []

    # Check each name in the 'Names' column
    for name in df['Names']:
        # Count for personal
        counts_p = sum(1 for value in personal if value in name)
        counts_personal.append(counts_p)

        # Count for comorbidities
        counts_c = sum(1 for value in comorbidities if value in name)
        counts_comorbidities.append(counts_c)

        # Count for reason for admit
        counts_r = sum(1 for value in reason_for_admit if value in name)
        counts_reason_admit.append(counts_r)

        # Count for medications
        counts_m = sum(1 for value in medications if value in name)
        counts_medications.append(counts_m)

    # Add new count columns to the DataFrame
    df['Counts_Personal'] = counts_personal
    df['Counts_Comorbidities'] = counts_comorbidities
    df['Counts_Reason_Admit'] = counts_reason_admit
    df['Counts_Medications'] = counts_medications

    return df

# Check values and update DataFrame
updated_df = check_value(summary_df)

updated_df

,Bin,Names,Counts_Personal,Counts_Comorbidities,Counts_Reason_Admit,Counts_Medications
0,10-20,"[id, hospital_length_of_stay, age, sex, height...",9,18,15,345
1,20-30,"[id, hospital_length_of_stay, age, sex, height...",9,16,8,162
2,30-40,"[id, hospital_length_of_stay, age, sex, height...",9,16,8,142
3,40-50,"[id, hospital_length_of_stay, age, sex, height...",9,13,3,59
4,50-60,"[id, hospital_length_of_stay, age, sex, height...",8,12,4,31
5,60-70,"[id, hospital_length_of_stay, age, sex, height...",9,5,2,8
6,70-80,"[id, hospital_length_of_stay, age, sex, height...",8,4,1,4


In [127]:
# Check values and get counts
updated_df1 = check_value(summary_df1)
updated_df2 = check_value(summary_df2)
updated_df3 = check_value(summary_df3)
updated_df4 = check_value(summary_df4)
updated_df5 = check_value(summary_df5)

In [134]:
updated_df5

,Bin,Names,Counts_Personal,Counts_Comorbidities,Counts_Reason_Admit,Counts_Medications
0,10-20,"[id, hospital_length_of_stay, age, sex, height...",9,13,8,100
1,20-30,"[id, hospital_length_of_stay, age, sex, height...",9,16,10,123
2,30-40,"[id, hospital_length_of_stay, age, sex, height...",9,18,10,214
3,40-50,"[id, hospital_length_of_stay, age, sex, height...",9,17,9,116
4,50-60,"[id, hospital_length_of_stay, age, sex, height...",9,14,8,55
5,60-70,"[id, hospital_length_of_stay, age, sex, height...",9,10,4,23
6,70-80,"[id, hospital_length_of_stay, age, sex, height...",8,4,1,4
